In [ ]:
import json
import calendar
from datetime import datetime, timedelta
from ics import Calendar, Event
from zoneinfo import ZoneInfo  # Python 3.9+

# Daten aus JSON laden
with open("dienstplan.json", "r", encoding="utf-8") as f:
    data = json.load(f)
monat = data["monat"]
jahr = data["jahr"]
dienstformen = {int(k): v for k, v in data["dienstformen"].items()}

def dienstzeiten(dienstform, datum):
    # Dienstzeiten und Wegezeiten
    zeiten = {
        "FD": ("06:00", "15:30"),  # 6:30 - 15:00 + 30min vorher/nachher
        "ZD": ("09:30", "19:30"),  # 10:00 - 19:00 + 30min vorher/nachher
        "SD": ("14:00", "23:30"),  # 14:30 - 23:00 + 30min vorher/nachher
        "ND": ("22:00", "08:30"),  # 22:30 - 08:00 + 30min vorher/nachher (Folgetag)
    }
    if dienstform == "Frei":
        return None, None
    tz = ZoneInfo("Europe/Berlin")
    start_str, end_str = zeiten[dienstform]
    start_dt = datetime.combine(datum, datetime.strptime(start_str, "%H:%M").time(), tzinfo=tz)
    # Nachtdienst endet am Folgetag
    if dienstform == "ND":
        end_dt = datetime.combine(datum + timedelta(days=1), datetime.strptime(end_str, "%H:%M").time(), tzinfo=tz)
    else:
        end_dt = datetime.combine(datum, datetime.strptime(end_str, "%H:%M").time(), tzinfo=tz)
    return start_dt, end_dt

def erstelle_kalender(monat, jahr, dienstformen):
    cal = Calendar()
    _, num_days = calendar.monthrange(jahr, monat)
    for tag in range(1, num_days + 1):
        datum = datetime(jahr, monat, tag).date()
        dienstform = dienstformen.get(tag, "Frei")
        start, end = dienstzeiten(dienstform, datum)
        if start and end:
            e = Event()
            e.name = f"{dienstform} Dienst"
            e.begin = start
            e.end = end
            e.description = f"{dienstform} am {datum.strftime('%d.%m.%Y')}"
            cal.events.add(e)
    return cal

# Kalender erstellen und als .ics Datei speichern
cal = erstelle_kalender(monat, jahr, dienstformen)
with open("dienstplan.ics", "w", encoding="utf-8") as f:
    f.write(str(cal))